<a href="https://colab.research.google.com/github/Abanoub1111/GBG_TASKS/blob/main/RAG_Overview/Task1_simple_rag_arabic_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q sentence-transformers faiss-cpu langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 73.0 MB/s eta 0:00:00


In [2]:
import re

def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "و", text)
    text = re.sub("ئ", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("[ًٌٍَُِْـ]", "", text)
    return text


In [3]:
from google.colab import files

uploaded = files.upload()

filename = list(uploaded.keys())[0]

with open(filename, "r", encoding="utf-8") as f:
    text = normalize_arabic(f.read())


Saving arabic.txt to arabic.txt


In [4]:
pip install langchain-text-splitters

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=100
)

chunks = splitter.split_text(text)

print("Number of chunks:", len(chunks))


Number of chunks: 9


In [6]:
from sentence_transformers import SentenceTransformer

embed_model = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


tokenizer_config.json:   0%|          | 0.00/526 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
doc_embeddings = embed_model.encode(
    chunks,
    normalize_embeddings=True,
    show_progress_bar=True
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
import faiss
import numpy as np

dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)

index.add(np.array(doc_embeddings))


In [9]:
def retrieve(query, k=4):
    query_embedding = embed_model.encode(
        query,
        normalize_embeddings=True
    )

    scores, indices = index.search(
        np.array([query_embedding]),
        k
    )

    return [chunks[i] for i in indices[0]]


In [11]:
question = "ماذا كان يسمى البرج الجنوبي سابقا"

results = retrieve(question)

for i, chunk in enumerate(results):
    print(f"\nResult {i+1}:")
    print(chunk)



Result 1:
ومع دخول المنطقه تحت مظله الحكم العثماني، شهدت القلعه انعطافات تاريخيه ووظيفيه حاده؛ فقد تحولت من مقر للسياده الي ثكنه عسكريه، ثم تلا ذلك تحويل "البرج الجنوبي"—الذي كان يعرف سابقا ببرج الرياح لدقه تصميمه في التهويه—الي معتقل سياسي شديد الحراسه خصص للمعارضين من الاعيان والولاه المتمردين. اما الساحات الشماليه الفسيحه، فقد اعيد توظيفها لتصبح "شوما" (مخازن) استراتيجيه لتخزين الحبوب والملح والبارود، وهو ما جعلها هدفا متكررا للنيران. ولقد عانت القلعه من نكبات متتاليه، ابرزها "زلزال القرن السابع عشر" والحريق الهايل الذي تلاه، مما ادي الي تصدع الاسوار الخارجيه وانهيار الاسقف الخشبيه المزخرفه، لتدخل

Result 2:
واليوم، تقف "القلعه البيضاء" شامخه كايقونه حضاريه ومتحف مفتوح يجتذب الاف الزوار والباحثين سنويا. لقد تم تحويل القاعات القديمه بذكاء هندسي الي صالات عرض متحفيه مكيفه تحفظ المقتنيات وفقا لاعلي المعايير العالميه، كما اعيد تاهيل الفناء الرييسي ليصبح مسرحا مكشوفا ذا خلفيه بانوراميه، يحتضن المهرجانات الموسيقيه والفعاليات الفنيه العالميه. هكذا، لم تعد القلعه مجرد اطلال صامته، بل اصبحت